In [1]:
import os
import json
import pickle
import numpy as np
import pandas as pd

In [2]:
files = ['misc_data/EFFICIENTDETD0_GuidedBackpropagation.txt', 'misc_data/EFFICIENTDETD0_IntegratedGradients.txt',
         'misc_data/EFFICIENTDETD0_SmoothGrad_GuidedBackpropagation.txt']
columns = ['image_index', 'object_index', 'class_name', 
           'class_confidence', 'coordinates', 'image_path',
           'mp_4', 'mp_13', 'mp_50', 'mp_100', 
           'eps_4', 'eps_13', 'eps_50', 'eps_100']

In [3]:
all_model_method = dict()
for i in files:
    model_name = os.path.basename(i)
    data = [json.loads(line) for line in open(i, 'r')]
    data = np.array(data)
    all_model_method[model_name.strip('.txt')] = data

/media/deepan/externaldrive1/project_repos/paz_versions/paz_efficientdet/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


In [4]:
list(all_model_method.keys())

['EFFICIENTDETD0_GuidedBackpropagation',
 'EFFICIENTDETD0_IntegratedGradients',
 'EFFICIENTDETD0_SmoothGrad_GuidedBackpropagation']

In [5]:
all_information_list = list()
for model_name, data in all_model_method.items():
    all_information = dict()
    all_information['name'] = model_name
    classes = np.unique(data[:, columns.index('class_name')], return_counts=True)[0]
    all_information['classes'] = list(classes)
    eps_cols = [n for n, i in enumerate(columns) if 'eps' in i]
    final_eps = []
    example_images = []
    for i in classes:
        class_data_idx = np.where(data[:, columns.index('class_name')] == i)
        class_data = data[class_data_idx[0], :]
        class_eps = class_data[:, eps_cols].astype('float32')
        mean_eps = np.mean(class_eps, axis=0)
        final_eps.append(list(mean_eps))
        example_images.append(list(class_data[:, columns.index('image_index')]))
    all_information['eps'] = final_eps
    all_information['example_images'] = example_images
    all_information_list.append(all_information)

In [8]:
save_file_name = 'misc_data/dbscan_hp.pkl'
with open(save_file_name, 'wb') as f:
    pickle.dump(all_information_list, f)

with open(save_file_name, 'rb') as f:
    read_data = pickle.load(f)
